###### <a id = 'r1'>
<font size = 6 color = 'midnightblue'>**CNN Based Malware Detection in backed-up images** 

### Decrypting and extracting the backup data [As per application this module will be changed]

In [ ]:
import pyzipper
import os

def decrypt_and_extract(zip_file, password, extract_path):
    with pyzipper.AESZipFile(zip_file) as zf:
        zf.pwd = password.encode()
        zf.extractall(path=extract_path)
    print(f"Files extracted to: {extract_path}")

# Example usage
zip_path = "D:\\Backup_MalwareChecker_Model\\BackupImage01.zip"
password = "MyBackupImage_Password"
extract_path = "D:\\Backup_MalwareChecker_Model\\BackupImage01_ExtractedFiles"

decrypt_and_extract(zip_path, password, extract_path)

### Converting Extracted Files to Images Format

In [25]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

def binary_to_image(file_path, img_size=(256, 256)):
    with open(file_path, 'rb') as f:
        binary_data = f.read()
    
    byte_array = np.frombuffer(binary_data, dtype=np.uint8)
    required_size = img_size[0] * img_size[1]

    if len(byte_array) < required_size:
        byte_array = np.pad(byte_array, (0, required_size - len(byte_array)), mode='constant')
    elif len(byte_array) > required_size:
        byte_array = byte_array[:required_size]
    
    img = np.reshape(byte_array, img_size)
    img = img / 255.0
    return img

file_path = "D:/Backup_MalwareChecker_Model/BackupImage01/ExportFromChart.csv"
image_data = binary_to_image(file_path)
cv2.imwrite("output_image.png", image_data * 255)

True

### Training CNN for Malware Detection

In [27]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = data_gen.flow_from_directory(
    "D:/Backup_MalwareChecker_Model/malimg_dataset/train/",
    target_size=(256, 256),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_generator = data_gen.flow_from_directory(
    "D:/Backup_MalwareChecker_Model/malimg_dataset/test/",
    target_size=(256, 256),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 5978 images belonging to 25 classes.
Found 181 images belonging to 25 classes.


### Defining CNN model

In [29]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

opt = Adam(learning_rate=0.001, decay=1e-6)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    29,491,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,585,857 (112.86 MB)

 Trainable params: 29,585,409 (112.86 MB)

 Non-trainable params: 448 (1.75 KB)

### Train CNN Model

In [31]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2
)

Epoch 1/2
187/187 ━━━━━━━━━━━━━━━━━━━━ 344s 2s/step - accuracy: 0.0129 - loss: -1941750.8750 - val_accuracy: 0.0110 - val_loss: 10416.4609
Epoch 2/2
187/187 ━━━━━━━━━━━━━━━━━━━━ 341s 2s/step - accuracy: 0.0120 - loss: -39395264.0000 - val_accuracy: 0.0110 - val_loss: -1731880.0000


### Deployment/Testing the model

In [36]:
def predict_file(file_path, model):
    img = binary_to_image(file_path)
    img = np.stack((img,) * 3, axis=-1)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)

    if prediction > 0.5:
        return "🚨 Malware Detected"
    else:
        return "✅ Clean File"

filepath = "D:/Backup_MalwareChecker_Model/BackupImage01/ExportFromChart.csv"
result = predict_file(filepath, model)
print(f"{result} for file {filepath}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step
✅ Clean File for file D:/Backup_MalwareChecker_Model/BackupImage01/ExportFromChart.csv
